<a href="https://colab.research.google.com/github/navidre91/Distributed_Submodular_Max/blob/master/Distributed_submodular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from numpy import linalg as LA
from numpy import random


A field that has 5*5 locations for sensor placement. which are distributed like a grid in the field. they are seprated 1 unit apart. Also the very left,right,top and bottom points are 1 unit apart from the edge of the field.
4 agents are to be placed in 4 of the nodes in the grid. 
There 10 events distributed randomly in the field. 

In [80]:
event_count=50
map_dim=5
agent_count=5
policy_count=agent_count*map_dim*map_dim
agent_sense_radius=np.array([1.3,2,2.3,0.7,1.6])

def event_generator(event_count,map_dim):
  event_location=np.empty((2,event_count))

  for i in range(event_count):
    event_location[0,i]=random.rand()*(map_dim+1)
    event_location[1,i]=random.rand()*(map_dim+1)
  return(event_location)

event_location=event_generator(event_count,map_dim)
print(event_location)


[[3.18297503e+00 4.30769594e+00 3.26363612e+00 1.58664622e+00
  4.36245311e-01 2.67858306e+00 3.71707750e+00 2.89712725e+00
  1.69549741e+00 3.25066521e+00 5.64011821e+00 1.49719724e+00
  3.56861245e+00 3.41667255e+00 2.99168178e+00 2.24365734e-03
  8.34211913e-01 4.33825215e+00 9.84324425e-01 2.58559024e+00
  3.46320639e+00 5.08296215e+00 5.48632805e+00 2.57299811e+00
  5.51952945e+00 5.27888857e-01 4.66019821e+00 3.16003075e+00
  2.65615139e+00 2.29888279e+00 2.80518451e+00 8.48492414e-01
  2.63680160e+00 1.47215969e+00 7.92089604e-01 2.32255445e+00
  4.06442057e-01 3.23085548e+00 8.40275196e-01 2.03684787e+00
  5.71883275e+00 1.84526776e+00 4.87566175e-01 5.77260147e+00
  1.96382231e+00 3.08887316e+00 2.58562196e+00 5.74220237e+00
  3.23708740e+00 3.29781471e+00]
 [3.02317458e+00 1.13393143e+00 1.71049590e+00 3.43180785e+00
  3.53605367e+00 5.51708333e+00 1.08268324e+00 7.60860345e-01
  9.29942690e-01 2.71045341e+00 2.45793405e+00 3.27869901e+00
  5.44952663e+00 7.64062248e-01 2.821

In [81]:
fig = go.Figure()

fig.update_layout(
    autosize=False,
    width=400,
    height=400,
    legend=dict(x=0.5, y=1),
    showlegend=False,
    font=dict(
        size=16,
        color="#7f7f7f",
        family="Courier New, monospace"
    ),
    template="simple_white",
    xaxis = dict(
        title = r'$x$',
        title_standoff = 0
    ),
    yaxis = dict(
        title = r'$y$',
        title_standoff = 0
    )
)

for i in range(map_dim+2):
  for j in range(map_dim+2):
    fig.add_trace(go.Scatter(
        x=[j],y=[i],
        line = dict(color='gray',width=1),
    ))

fig.add_trace(go.Scatter(
    x=event_location[0,:],y=event_location[1,:],
    mode='markers',
    line = dict(color='red',width=1),
  ))



fig.show()

In [82]:
def policy2map(policy_num,map_dim):
  policy_num_agent_wise=(policy_num-1)%(map_dim*map_dim)+1
  y=int((policy_num_agent_wise-1)/map_dim)+1
  x=(policy_num_agent_wise-1)%map_dim+1
  coordination=np.array([[x],[y]])
  return coordination

def coverage(policy_vect,agent_sense_radius,event_location,map_dim):
  count=0
  policy_count=len(policy_vect)
  agent_count=len(agent_sense_radius)
  event_count=event_location.shape[1]
  event_taken_ind = np.zeros((1,event_count))

  for i in range(policy_count):
    if policy_vect[i]==1:
      agent = int(i/(map_dim*map_dim))
      loc = policy2map(i+1,map_dim)
      for j in range(event_count):
        if event_taken_ind[0,j]==0:
          if np.sqrt (  (loc[0]-event_location[0,j])*(loc[0]-event_location[0,j])  + (loc[1]-event_location[1,j])*(loc[1]-event_location[1,j]) ) < agent_sense_radius[agent]:
            event_taken_ind[0,j]=1
            count=count+1
  return count


In [83]:
def rand_policy_gen(policy_prob_vect,sample_count):
  policy_count = len(policy_prob_vect)
  sampled_policies = np.zeros((policy_count,sample_count))
  for i in range(policy_count):
    prob=policy_prob_vect[i]
    for j in range(sample_count):
      rand=random.rand()
      if prob>=rand:
        sampled_policies[i,j]=1
  return sampled_policies


def stoch_gradient(policy_prob_vect,sample_count,agent_sense_radius,event_location,map_dim):
  policy_count=len(policy_prob_vect)
  stoch_gradient_vect=np.zeros((policy_count,1)) 
  policy_samples=rand_policy_gen(policy_prob_vect,sample_count)
  for i in range(policy_count):
    sum=0
    for j in range(sample_count):
      sample = (policy_samples[:,j])
      sample_inc=np.copy(sample)
      sample_inc[i]=1
      sample_exc=np.copy(sample)
      sample_exc[i]=0
      sum=sum+coverage(sample_inc,agent_sense_radius,event_location,map_dim)-coverage(sample_exc,agent_sense_radius,event_location,map_dim)
    stoch_gradient_vect[i]=float(sum/sample_count)
  return stoch_gradient_vect

def max_vect(vects):
  n=vects.shape[0]
  m=vects.shape[1]
  max_v=np.zeros((n,))
  for i in range(n):
    max=0
    for j in range(m):
      if vects[i,j]>max:
        max=vects[i,j]
    max_v[i]=max
  return max_v

def max_grad_agent_wise(sto_grad,agent,agent_count,map_dim):
  policy_count=agent_count*map_dim*map_dim
  max=0
  ind=-1
  for i in range(agent*(map_dim*map_dim),(agent+1)*(map_dim*map_dim)):
    if sto_grad[i]>max:
      max=sto_grad[i]
      ind=i
  return ind

In [ ]:
sample_count=100
T=100

x=np.zeros((policy_count,agent_count))
agent_sense_radius=np.array([1.3,2,2.3,0.7,1.6])
for i in range(T):
  print(i)
  for j in range (agent_count):
    sto_g=stoch_gradient(x[:,j],sample_count,agent_sense_radius,event_location,map_dim)
    ind=max_grad_agent_wise(sto_g,j,agent_count,map_dim)
    x[ind,j]=x[ind,j]+1/T
  x[:,0]=max_vect( np.copy((x[:,[0,1,2]])))
  x[:,1]=max_vect( np.copy((x[:,[0,1,3]])))
  x[:,2]=max_vect( np.copy((x[:,[0,2]])))
  x[:,3]=max_vect( np.copy((x[:,[1,3,4]])))
  x[:,4]=max_vect( np.copy((x[:,[3,4]])))
  

In [ ]:
print(x)